In [7]:
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 6.1 MB/s eta 0:00:0000:0100:01


In [8]:
# Cell 1: 필요한 라이브러리 임포트
import sys
import os
sys.path.insert(0, '/home/grey1/stock-kafka3/airflow/plugins')
sys.path.insert(0, '/home/grey1/stock-kafka3/airflow/common')

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time
import duckdb

# Cell 2: DuckDBManager 클래스 직접 정의 (임포트 에러 해결)
class DuckDBManager:
    """DuckDB 데이터베이스 관리 클래스"""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.con = duckdb.connect(database=db_path)
        self._initialize_tables()
    
    def _initialize_tables(self):
        """테이블 초기화"""
        # nasdaq_symbols 테이블
        self.con.execute("""
            CREATE TABLE IF NOT EXISTS nasdaq_symbols (
                symbol VARCHAR PRIMARY KEY,
                name VARCHAR,
                market_cap BIGINT,
                sector VARCHAR,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        # stock_data 테이블
        self.con.execute("""
            CREATE TABLE IF NOT EXISTS stock_data (
                symbol VARCHAR,
                date DATE,
                open DOUBLE,
                high DOUBLE,
                low DOUBLE,
                close DOUBLE,
                volume BIGINT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                PRIMARY KEY (symbol, date)
            )
        """)
    
    def get_active_symbols(self):
        """활성 심볼 목록 조회"""
        try:
            result = self.con.execute("""
                SELECT symbol FROM nasdaq_symbols
            """).fetchall()
            
            # 튜플 리스트를 문자열 리스트로 변환
            symbols = [row[0] for row in result]
            return symbols
        except Exception as e:
            print(f"❌ 심볼 조회 오류: {e}")
            return []
    
    def save_stock_data(self, data: dict):
        """주가 데이터 저장"""
        try:
            self.con.execute("""
                INSERT OR REPLACE INTO stock_data 
                (symbol, date, open, high, low, close, volume)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                data['symbol'],
                data['date'],
                data['open'],
                data['high'],
                data['low'],
                data['close'],
                data['volume']
            ))
            return True
        except Exception as e:
            print(f"❌ 데이터 저장 오류: {e}")
            return False
    
    def close(self):
        """연결 종료"""
        if hasattr(self, 'con'):
            self.con.close()

# Cell 3: YFinanceCollector 클래스 정의
class YFinanceCollector:
    """yfinance 기반 주식 데이터 수집 클래스 (테스트용)"""
    
    def __init__(self, db_path: str = "/home/grey1/stock-kafka3/data/duckdb/stock_data.db"):
        self.db = DuckDBManager(db_path)
    
    def collect_stock_data(self, symbol: str, period: str = "1mo") -> bool:
        """개별 종목 주가 데이터 수집 (테스트용 - 1개월)"""
        import random
        
        try:
            # API 호출 제한 방지를 위한 지연
            delay = random.uniform(0.5, 1.5)
            time.sleep(delay)
            
            print(f"🔍 {symbol} 데이터 수집 중...")
            
            # yfinance로 데이터 수집
            ticker = yf.Ticker(symbol)
            hist = ticker.history(
                period=period, 
                auto_adjust=True,
                prepost=False,
                actions=False,
                repair=True
            )
            
            if hist.empty:
                print(f"⚠️ {symbol}: 데이터가 비어있음")
                return False
            
            print(f"✅ {symbol}: {len(hist)}개 레코드 수집됨")
            
            # 데이터 저장 (선택사항)
            # hist = hist.reset_index()
            # for _, row in hist.iterrows():
            #     self.db.save_stock_data({
            #         'symbol': symbol,
            #         'date': row['Date'].date(),
            #         'open': row['Open'],
            #         'high': row['High'],
            #         'low': row['Low'],
            #         'close': row['Close'],
            #         'volume': row['Volume']
            #     })
            
            return True
            
        except Exception as e:
            print(f"❌ {symbol}: 수집 실패 - {e}")
            return False
    
    def close(self):
        """리소스 정리"""
        if hasattr(self, 'db'):
            self.db.close()

# Cell 4: 단일 종목 테스트
collector = YFinanceCollector()

# 애플 주식 테스트
test_symbol = "AAPL"
print(f"📊 {test_symbol} 주식 데이터 수집 테스트")

success = collector.collect_stock_data(test_symbol, period="5d")  # 5일치만
print(f"결과: {'성공' if success else '실패'}")

# Cell 5: yfinance 직접 테스트 (디버깅용)
ticker = yf.Ticker("AAPL")
hist = ticker.history(period="5d")

print(f"📊 수집된 데이터 shape: {hist.shape}")
print(f"📊 컬럼: {hist.columns.tolist()}")
print("\n📊 최근 5일 데이터:")
print(hist.tail())

# Cell 6: 심볼 형식 문제 디버깅
db = DuckDBManager("/home/grey1/stock-kafka3/data/duckdb/stock_data.db")
symbols = db.get_active_symbols()

print(f"🔍 DB에서 조회된 심볼 타입: {type(symbols)}")
print(f"🔍 심볼 개수: {len(symbols) if symbols else 0}")

if symbols and len(symbols) > 0:
    print(f"\n🔍 첫 5개 심볼:")
    for i, symbol in enumerate(symbols[:5]):
        print(f"  [{i}] 타입: {type(symbol)}, 값: '{symbol}'")

db.close()

📊 AAPL 주식 데이터 수집 테스트
🔍 AAPL 데이터 수집 중...
✅ AAPL: 5개 레코드 수집됨
결과: 성공
📊 수집된 데이터 shape: (5, 7)
📊 컬럼: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']

📊 최근 5일 데이터:
                                 Open        High         Low       Close  \
Date                                                                        
2025-07-14 00:00:00-04:00  209.929993  210.910004  207.539993  208.619995   
2025-07-15 00:00:00-04:00  209.220001  211.889999  208.919998  209.110001   
2025-07-16 00:00:00-04:00  210.300003  212.399994  208.639999  210.160004   
2025-07-17 00:00:00-04:00  210.570007  211.800003  209.589996  210.020004   
2025-07-18 00:00:00-04:00  210.869995  211.789993  209.699997  211.179993   

                             Volume  Dividends  Stock Splits  
Date                                                          
2025-07-14 00:00:00-04:00  38840100        0.0           0.0  
2025-07-15 00:00:00-04:00  42296300        0.0           0.0  
2025-07-16 00:00:00-04:00 